# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1616461074990_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col
from pyspark.sql import *

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-spring2021"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [3]:
def user():
    # Returns a string consisting of your GT username.
    return 'yraparia3'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [4]:
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
    trips =trips.filter(col("trip_distance")>=2)
    return trips

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [5]:
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
    #var df3=pyspark.emptyDataFrame;

    df3=trips.join(lookup,trips.DOLocationID ==  lookup.LocationID,"inner")
    df3=df3.withColumn("pcount",col("passenger_count").cast("int"))
    df3=df3.filter(col("Borough")=="Manhattan")
    df3=df3.groupBy(col("LocationID")).sum("pcount")
    df3=df3.select(col("LocationID").alias("DOLocationID"),col("sum(pcount)").alias("pcount"))
    df3=df3.orderBy(df3["pcount"],ascending=False).limit(20)

    return df3
    


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [6]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    trips=trips.withColumn("f_amount",col("total_amount").cast("double"))
    top_DO = mtrips.select("DOLocationID").collect()
    topDO = [row.DOLocationID for row in top_DO]


    #df5=trips.filter(trips.DOLocationID.isin(topDO))
    df5=trips.groupBy(col("PULocationID"),col("DOLocationID")).count().filter(trips.DOLocationID.isin(topDO))
    df5=df5.groupBy(col("PULocationID")).sum("count")
    df5=df5.select(col("PULocationID"),col("sum(count)").alias("countp"))
    df5=df5.select(col("PULocationID").alias("POLocID"),col("countp"))

    df10=trips.groupBy(col("PULocationID")).avg("f_amount")
    df10=df10.select(col("PULocationID"),col("avg(f_amount)").alias("average total amount"))
    df11=trips.groupBy(col("PULocationID")).count()
    df11=df11.select(col("PULocationID").alias("pid"),col("count").alias("count"))
    df4=df10.join(df11,df10.PULocationID==df11.pid,"rightouter")

    df6=df4.join(df5,df4.PULocationID==df5.POLocID,"leftouter")
    df6.na.fill(value=0,subset=["countp"])
    df6=df6.select(col("PULocationID"),(col("countp")/col("count")*col("average total amount")).cast("decimal(10,8)").alias("weighted_profit"))

    return df6




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [7]:

def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    dff = calc.join(lookup, lookup.LocationID==calc.PULocationID,"inner")
    dff = dff.select(col("Zone"),col("Borough"),col("weighted_profit"))
    dff = dff.orderBy(dff["weighted_profit"], ascending=False).limit(20)
    return dff
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the path to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear in a folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [9]:
# update your bucket path
bucket = 's3://cse6242-yraparia3/e-2VOSAVLWWLSP6RDG9VTL1ED40/q3_output_small'
main('small', bucket)

bucket = 's3://cse6242-yraparia3/e-2VOSAVLWWLSP6RDG9VTL1ED40/q3_output_large'
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

yraparia3
Trip Count:  7667792
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|Arrochar/Fort Wad...|Staten Island|    31.78444444|
|     Freshkills Park|Staten Island|    29.34250000|
|       Port Richmond|Staten Island|    24.35777778|
|Flushing Meadows-...|       Queens|    24.33013773|
|        Baisley Park|       Queens|    23.62469433|
|     Randalls Island|    Manhattan|    23.12457752|
|       South Jamaica|       Queens|    22.17411494|
|     Mariners Harbor|Staten Island|    21.78655693|
|Saint George/New ...|Staten Island|    21.49189349|
|       Arden Heights|Staten Island|    20.29625000|
|        Astoria Park|       Queens|    19.96536000|
|   LaGuardia Airport|       Queens|    17.92315395|
|  Murray Hill-Queens|       Queens|    17.58545042|
|         JFK Airport|       Queens|    16.95897820|
|             Jamaica|       Queens|    14.87983519|
|Governor's Isl

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>

In [12]:
print(user())
bucket = 's3://cse6242-yraparia3/e-2VOSAVLWWLSP6RDG9VTL1ED40/q3_output'
trips, lookup = load_data(size='small')
trips = long_trips(trips)
mtrips = manhattan_trips(trips, lookup)
wp = weighted_profit(trips, mtrips)
wp.show(30)
#df6.show(30)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

yraparia3
Trip Count:  7667792
+------------+---------------+
|PULocationID|weighted_profit|
+------------+---------------+
|         148|     7.69740615|
|         243|     7.93404251|
|          65|     6.63069823|
|         133|     2.91508945|
|         126|     5.50219539|
|          81|     4.00553633|
|          76|     3.65623752|
|         192|     3.40770858|
|          12|    10.55403660|
|         209|     9.29151592|
|         122|     7.36253058|
|         140|     8.28884629|
|         177|     2.19967393|
|           1|     4.24235102|
|         206|    21.49189349|
|           6|    31.78444444|
|         205|     6.96914892|
|           3|     5.46186343|
|         142|     7.90567236|
|         178|     4.46143008|
|         250|     4.28319668|
|         191|     5.22753284|
|          92|     3.57543019|
|          64|     0.88654956|
|         112|     3.80907283|
|         189|     3.87357477|
|         207|    13.17240115|
|          37|     2.30075390|
|       

In [13]:
final = final_output(wp,lookup)

# Outputs the results for you to visually see
final.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|Arrochar/Fort Wad...|Staten Island|    31.78444444|
|     Freshkills Park|Staten Island|    29.34250000|
|       Port Richmond|Staten Island|    24.35777778|
|Flushing Meadows-...|       Queens|    24.33013773|
|        Baisley Park|       Queens|    23.62469433|
|     Randalls Island|    Manhattan|    23.12457752|
|       South Jamaica|       Queens|    22.17411494|
|     Mariners Harbor|Staten Island|    21.78655693|
|Saint George/New ...|Staten Island|    21.49189349|
|       Arden Heights|Staten Island|    20.29625000|
|        Astoria Park|       Queens|    19.96536000|
|   LaGuardia Airport|       Queens|    17.92315395|
|  Murray Hill-Queens|       Queens|    17.58545042|
|         JFK Airport|       Queens|    16.95897820|
|             Jamaica|       Queens|    14.87983519|
|Governor's Island...|    Manhattan|    14.796